In [ ]:
!pip install tensorflowjs

In [ ]:
import os
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from kaggle_secrets import UserSecretsClient
import tensorflowjs as tfjs

In [ ]:
# 1. Persiapkan Dataset
from kaggle_secrets import UserSecretsClient
# Pengaturan Kaggle
user_secrets = UserSecretsClient()
KAGGLE_KEY = user_secrets.get_secret("KAGGLE_KEY")
KAGGLE_USERNAME = user_secrets.get_secret("KAGGLE_USERNAME")
os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
os.environ["KAGGLE_KEY"] = KAGGLE_KEY

In [ ]:
# Unduh dan ekstrak dataset
!kaggle datasets download -d andrewmvd/animal-faces
!unzip -o -q animal-faces.zip


In [ ]:
# Gabungkan dataset train dan val
train_dir = "afhq/train"
val_dir = "afhq/val"
combined_dir = "animal/dataset"
os.makedirs(combined_dir, exist_ok=True)

In [ ]:
for category in os.listdir(train_dir):
    category_dir = os.path.join(train_dir, category)
    if os.path.isdir(category_dir):
        shutil.copytree(category_dir, os.path.join(combined_dir, category), dirs_exist_ok=True)

for category in os.listdir(val_dir):
    category_dir = os.path.join(val_dir, category)
    if os.path.isdir(category_dir):
        shutil.copytree(category_dir, os.path.join(combined_dir, category), dirs_exist_ok=True)

In [ ]:
# Visualisasi data
animal_image = {}
path_sub = os.path.join("animal", "dataset")
for category in os.listdir(path_sub):
    category_path = os.path.join(path_sub, category)
    if os.path.isdir(category_path):
        animal_image[category] = os.listdir(category_path)

fig, axs = plt.subplots(len(animal_image.keys()), 5, figsize=(15, 10))
for i, class_name in enumerate(animal_image.keys()):
    images = np.random.choice(animal_image[class_name], 5, replace=False)
    for j, image_name in enumerate(images):
        img_path = os.path.join(path_sub, class_name, image_name)
        img = plt.imread(img_path)
        axs[i, j].imshow(img)
        axs[i, j].set(xlabel=class_name, xticks=[], yticks=[])
        axs[i, j].xaxis.set_label_position('top')
fig.tight_layout()
plt.show()


In [ ]:
# Distribusi kelas
animal_path = "animal/dataset"
file_name = []
labels = []
full_path = []

for path, subdirs, files in os.walk(animal_path):
    for name in files:
        full_path.append(os.path.join(path, name))
        labels.append(os.path.basename(path))
        file_name.append(name)

distribution_train = pd.DataFrame({"path": full_path, "file_name": file_name, "labels": labels})

plt.figure(figsize=(8, 6))
sns.countplot(x=distribution_train["labels"])
plt.title("Distribusi Gambar per Kelas")
plt.xlabel("Kelas")
plt.ylabel("Jumlah Gambar")
plt.xticks(rotation=45)
plt.show()

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = len(os.listdir(animal_path))

In [ ]:
df = distribution_train[["path", "labels"]]
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df["labels"], random_state=123)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["labels"], random_state=123)

In [ ]:
# ImageDataGenerator untuk augmentasi
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="path",
    y_col="labels",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_dataframe(
    val_df,
    x_col="path",
    y_col="labels",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_dataframe(
    test_df,
    x_col="path",
    y_col="labels",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# 2. Bangun Model Sequential dengan Conv2D dan MaxPooling2D
model = models.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
# Kompilasi model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 3. Latih Model dengan Callback
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=3),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy')
]

In [ ]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=callbacks
)

In [ ]:
# 4. Evaluasi Model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Akurasi Test: {test_accuracy*100:.2f}%")

In [ ]:
# Confusion matrix dan classification report
y_true = test_generator.classes
y_pred = np.argmax(model.predict(test_generator), axis=1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Prediksi')
plt.ylabel('Sebenarnya')
plt.show()

print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

In [ ]:
# Plot akurasi dan loss
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(history.history['accuracy'], label='Akurasi Training')
ax1.plot(history.history['val_accuracy'], label='Akurasi Validasi')
ax1.set_title('Akurasi Training dan Validasi')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Akurasi')
ax1.legend()

ax2.plot(history.history['loss'], label='Loss Training')
ax2.plot(history.history['val_loss'], label='Loss Validasi')
ax2.set_title('Loss Training dan Validasi')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Simpan model
model.save("animal_model.keras")

In [ ]:
# Load model dari file .keras
model = tf.keras.models.load_model("animal_model.keras")

# Ambil satu batch dari test_generator
for images, labels in test_generator:
    if len(images) < 5:
        random_indices = list(range(len(images)))
    else:
        random_indices = random.sample(range(len(images)), 5)
    break  # Ambil satu batch saja

# Buat figure untuk menampilkan 5 gambar secara horizontal
plt.figure(figsize=(15, 5))

for i, idx in enumerate(random_indices):
    test_image = images[idx]
    true_label = list(test_generator.class_indices.keys())[np.argmax(labels[idx])]

    test_image_expanded = np.expand_dims(test_image, axis=0)
    prediction = model.predict(test_image_expanded)
    predicted_class = list(test_generator.class_indices.keys())[np.argmax(prediction[0])]

    # Tampilkan gambar
    plt.subplot(1, 5, i+1)
    plt.imshow(test_image)
    plt.title(f"Asli: {true_label}\nPred: {predicted_class}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Buat folder penyimpanan
os.makedirs("submissions/tfjs_model", exist_ok=True)
os.makedirs("submissions/tflite", exist_ok=True)
os.makedirs("submissions/saved_model", exist_ok=True)

# 1. Simpan ke format TensorFlow SavedModel
model.export("submissions/saved_model")

# 2. Simpan ke format TensorFlow.js
tfjs.converters.save_keras_model(model, "submissions/tfjs_model")

# 3. Simpan ke format TFLite
# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFLite
with open("submissions/tflite/model.tflite", "wb") as f:
    f.write(tflite_model)

# Simpan label (kelas)
labels = list(test_generator.class_indices.keys())
with open("submissions/tflite/label.txt", "w") as f:
    for label in labels:
        f.write(label + "\n")

In [ ]:
import shutil

# Buat zip dari folder submissions
shutil.make_archive('/kaggle/working/submissions', 'zip', '/kaggle/working/submissions')

In [ ]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/submissions/saved_model/saved_model.pb')